# JUICE RPWI HF SID4 (Burst surv): L1a QL -- 2023/10/29

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_sid4_lib as juice_sid4
# import juice_math_lib as juice_math

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
unit_mode = 0                           # 0: sum    1: /Hz
cal_mode = 0                            # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5

# *** Conversion factor: cal_mode ***
cf, p_max, p_min = juice_cdf.cal_factors(unit_mode, cal_mode, cal, p_raw_max, p_raw_min)

In [ ]:
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder
data_dir = '/Users/user/0-python/JUICE_data/'        # CDF data folder

# *** Ver.2 ***
# 202310 -- SAMPLE
data_name = 'test-CCSDS/sample/cdf/SID04_20231024-0042.cdf'

# get CDF data

In [ ]:
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid4.juice_getdata_hf_sid4(cdf)
date = data.epoch[0]
month = date.strftime('%Y%m')
print(data.epoch[0], "-", data.epoch[-1], "[", month, "]")

data.EuEu = data.EuEu * 10**(cf/10)
data.EvEv = data.EvEv * 10**(cf/10)
data.EwEw = data.EwEw * 10**(cf/10)
data.EuEu_dB = 10.0 * np.log10(data.EuEu)
data.EvEv_dB = 10.0 * np.log10(data.EvEv)
data.EwEw_dB = 10.0 * np.log10(data.EwEw)

n_freq = data.EuEu.shape[1]
n_time = data.EuEu.shape[0]
n_time0 = n_time-1
print("data size:", data.EuEu.shape, n_time, n_freq)

In [ ]:
# Mode 
N_ch = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
if (data.complex[0] == 0):
    print("complex:", data.complex[0], " (power 3-ch) <Table 6.3.2>")
    N_component = 3
elif (data.complex[0] == 1):
    print("complex:", data.complex[0], " (spectral matrix 3-ch) <Table 6.3.3>")
    N_component = 9
else:
    print("complex:", data.complex[0], " (3D spectral matrix 3-ch) <Table 6.3.4>")
    N_component = 21
print("Ch:", N_ch, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
N_frequency = data.B0_step[0] * data.B0_subdiv[0]
print("==> Component", N_component, "   Num-frequency", N_frequency, "  Num-sweep:", n_time)

# Data for checks

In [ ]:
Epoch_1d       = data.epoch.tolist()
freq_1d        = data.frequency[0].tolist()
freq1          = freq_1d[0:N_frequency]
freq_width_1d  = data.freq_width[0].tolist()
freq_width1    = freq_width_1d[0:N_frequency]
freq_width_2d  = data.freq_width.transpose()

EuEu_2d = data.EuEu.transpose()
EvEv_2d = data.EvEv.transpose()
EwEw_2d = data.EwEw.transpose()
Eu_power1 = data.EuEu[0][0:N_frequency]
Ev_power1 = data.EvEv[0][0:N_frequency]
Ew_power1 = data.EwEw[0][0:N_frequency]
Eu_power2 = data.EuEu[n_time0][0:N_frequency]
Ev_power2 = data.EvEv[n_time0][0:N_frequency]
Ew_power2 = data.EwEw[n_time0][0:N_frequency]
if unit_mode == 1:
    EuEu_2d = EuEu_2d / freq_width_2d / 1000
    EvEv_2d = EvEv_2d / freq_width_2d / 1000
    EwEw_2d = EwEw_2d / freq_width_2d / 1000
    Eu_power1 = Eu_power1 / freq_width1 / 1000
    Ev_power1 = Ev_power1 / freq_width1 / 1000
    Ew_power1 = Ew_power1 / freq_width1 / 1000
    Eu_power2 = Eu_power2 / freq_width1 / 1000
    Ev_power2 = Ev_power2 / freq_width1 / 1000
    Ew_power2 = Ew_power2 / freq_width1 / 1000
    print("frequency width:", freq_width1)

f_min = freq_1d[0]
f_max = freq_1d[N_frequency-1]
power_str = juice_cdf.power_label(cal_mode, unit_mode)

# Raw data

In [ ]:
fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1)
ax2 = fig.add_subplot(4, 1, 2)
ax3 = fig.add_subplot(4, 1, 3)
ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(data.EuEu), '-r', linewidth=.5)
ax1.plot(np.ravel(data.EvEv), '-g', linewidth=.5)
ax1.plot(np.ravel(data.EwEw), '-b', linewidth=.5)
ax1.set_yscale('log')
ax2.plot(np.ravel(data.EuEv_re), '-r', linewidth=.5)
ax2.plot(np.ravel(data.EvEw_re), '-g', linewidth=.5)
ax2.plot(np.ravel(data.EwEu_re), '-b', linewidth=.5)
ax2.plot(np.ravel(data.EuEv_im), ':r', linewidth=.5)
ax2.plot(np.ravel(data.EvEw_im), ':g', linewidth=.5)
ax2.plot(np.ravel(data.EwEu_im), ':b', linewidth=.5)
ax2.set_yscale('log')
ax3.plot(np.ravel(data.frequency), '-r', linewidth=.5)
ax3.plot(np.ravel(data.freq_step*1000), '-g', linewidth=.5)
ax3.plot(np.ravel(data.freq_width*1000), '-b', linewidth=.5)
ax4.plot(np.ravel(data.epoch), '.')
ax1.set_ylabel('Power [RAW @ ADC]')
ax2.set_ylabel('Real & Img [RAW @ ADC]')
ax3.set_ylabel('frequency [kHz]')
ax4.set_ylabel('Date-Time')
date = data.epoch[0]
date = date.strftime('%Y/%m/%d')
ax4.set_xlabel(date)

# range: X-axis
"""
xlim=[0, N_frequency*2-1]
ax1.set_xlim(xlim)
ax2.set_xlim(xlim)
ax3.set_xlim(xlim)
"""
# range: Y-axis
ylim=[10**p_raw_min, 10**p_raw_max]
ax1.set_ylim(ylim)
ax2.set_ylim(ylim)
ylim=[f_min, f_max]
ax3.set_ylim(ylim)

# Plot
fig.show
png_fname = work_dir+data_name+'_raw.png'
# fig.savefig(png_fname)

## Spec LOG

In [ ]:
# Y-range
p_min0 = p_min
p_max0 = p_max

fig, cal = plt.subplots(figsize=(12, 6))
cal.plot(freq1, Eu_power1, ':r', linewidth=.5)
cal.plot(freq1, Ev_power1, ':g', linewidth=.5)
cal.plot(freq1, Ew_power1, ':b', linewidth=.5)
cal.plot(freq1, Eu_power2, '-r', linewidth=.5)
cal.plot(freq1, Ev_power2, '-g', linewidth=.5)
cal.plot(freq1, Ew_power2, '-b', linewidth=.5)

cal.set_xlabel('Frequency [kHz]')
cal.set_xscale('log')
cal.set_xlim(f_min, f_max)

cal.set_yscale('log')
plt.ylabel(power_str)
cal.set_ylim(10**p_min0, 10**p_max0)

plt.show
png_fname = work_dir+data_name+'_spec-log.png'
# fig.savefig(png_fname)

## Spec-linear

In [ ]:
# Y-range
p_min0 = p_min
p_max0 = p_max

fig, cal = plt.subplots(figsize=(12, 6))
cal.plot(freq1, Eu_power1, ':r', linewidth=.5)
cal.plot(freq1, Ev_power1, ':g', linewidth=.5)
cal.plot(freq1, Ew_power1, ':b', linewidth=.5)
cal.plot(freq1, Eu_power2, '-r', linewidth=.5)
cal.plot(freq1, Ev_power2, '-g', linewidth=.5)
cal.plot(freq1, Ew_power2, '-b', linewidth=.5)

cal.set_xlabel('Frequency [kHz]')
cal.set_xlim(f_min, f_max)

cal.set_yscale('log')
plt.ylabel(power_str)
cal.set_ylim(10**p_min0, 10**p_max0)

plt.show
png_fname = work_dir+data_name+'_spec.png'
# fig.savefig(png_fname)

## FT - LOG

In [ ]:
p_min0 = p_min
p_max0 = p_max

fig2d = plt.figure(figsize=[12,10])
ax1 = fig2d.add_subplot(3, 1, 1)
ax2 = fig2d.add_subplot(3, 1, 2)
ax3 = fig2d.add_subplot(3, 1, 3)

# Y-axis
ax1.set_ylim(f_min, f_max)
ax2.set_ylim(f_min, f_max)
ax3.set_ylim(f_min, f_max)
ax1.set_yscale('log')
ax2.set_yscale('log')
ax3.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]')
ax2.set_ylabel('Frequency [kHz]')
ax3.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title('EuEu (x)')
ax2.set_title('EvEv (y)')
ax3.set_title('EwEw (z)')
date = data.epoch[0]
date = date.strftime('%Y/%m/%d')
ax3.set_xlabel(date)

# Plot
p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical")
pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical")
pp3.set_label(power_str)

plt.show()
png_fname = work_dir+data_name+'_FT-log.png'
# fig2d.savefig(png_fname)


## FT - linear

In [ ]:
p_min0 = p_min
p_max0 = p_max

fig2d = plt.figure(figsize=[12,10])
ax1 = fig2d.add_subplot(3, 1, 1)
ax2 = fig2d.add_subplot(3, 1, 2)
ax3 = fig2d.add_subplot(3, 1, 3)

# Y-axis
ax1.set_ylim(f_min, f_max)
ax2.set_ylim(f_min, f_max)
ax3.set_ylim(f_min, f_max)
ax1.set_ylabel('f [kHz]')
ax2.set_ylabel('f [kHz]')
ax3.set_ylabel('f [kHz]')

# X-axis
ax1.set_title('EuEu (X)')
ax2.set_title('EvEv (Z)')
ax3.set_title('EwEw (Y)')
date = data.epoch[0]
date = date.strftime('%Y/%m/%d')
ax3.set_xlabel(date)

# Plot
p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical")
pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical")
pp3.set_label(power_str)

plt.show()
png_fname = work_dir+data_name+'_FT.png'
# fig2d.savefig(png_fname)